In [2]:
import json
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nq-distilbert-base-v1')

taglist = []
questionlist = []
answerdict = {}

with open('intents_one.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    for qanda in data['intents']:
        taglist.append(qanda['tag'])
        for q in qanda['questions']:
            questionlist.append((qanda['tag'], q))
        for a in qanda['answers']:
            answerdict.update({qanda['tag']: a})

In [3]:
questionembeddinglist = [(tagandquestion[0], model.encode(tagandquestion[1])) for tagandquestion in questionlist]

def retrievesimilarities(query):
    similarities = []
    queryembedding = model.encode(query)
    for tagquestiontup in questionembeddinglist:
        similarities.append((util.pytorch_cos_sim(queryembedding, tagquestiontup[1]).item(), tagquestiontup[0]))
    return(max(similarities))  

In [11]:
def findanswer(query):
    similaritytagtup = retrievesimilarities(query)
    tag = similaritytagtup[1]
    return answerdict[tag]

In [20]:
#evaluation
table = []

with open('intents_test.json', encoding='utf-8-sig') as testfile:
    testdata = json.load(testfile)
    for qanda in testdata['intents']:
        liste = []
        query = qanda['questions'][0]
        liste.append(query)
        predictedanswer = str(findanswer(qanda['questions'][0]))
        liste.append(predictedanswer)
        realanswer = qanda['answers'][0]
        liste.append(realanswer)
        table.append(liste)

matches = 0
fails = 0
for row in table: 
    if row[1] == row[2]:
        matches += 1
    else: 
        fails += 1

success = 100 - (fails/(fails+matches))*100
print('The bot found the fitting answer for', success, '% of the test queries.')

The bot found the fitting answer for 76.19047619047619 % of the test queries.


In [33]:
#the bot at work, to try it out, run this cell again
print('Ask me anything about alcohol addiction! Type "quit" to exit')

while True: 
    query = input('You: ')
    if query == 'quit':
        break
    else:
        print(f'\nBot: {findanswer(query)}\n')

Ask me anything about alcohol addiction! Type "quit" to exit
You: hey

Bot: Hi, how can I help?

You: i am not sure if i have a drinking problem or not

Bot: The level of how much alcohol is too much varies from person to person and situation to situation. How severly alcohol is adversely affecting one's life can also vary over one's lifetime. One way to make this assessment is to consult with someone one is close to, perhaps friend or family member, or of course professionals are there for this reason. They can be more objective about one's alcohol use and its consequences. Note that it’s easy to underestimate the amount of alcohol one consumes daily or weekly. Especially in the setting of consistent drinking, it can also be hard to gauge whether one has developed significant physical dependence until attempts are made to cut back or quit. Others can judge one's consumption more accurately. Signs of addiction include: loss of control, exceeding the limits you’ve set for yourself, drin